.py dsoyaları: bütün dosyalar  
requirements.txt: versiyonlar  
readme.txt: kodu anlatacak  

### Rapor:
= Gizli katman sayısı nasıl etkiliyor?  
= Katmanlardaki nöron sayısı nasıl etkiliyor?  
= Learning rate error'u ve run time'ı nasıl etkiliyor?  
= Optimal Epoch sayısının nasıl hesaplanacağı?  
= Her bir parametre error'u nasıl etkiliyor?  

### Teslim
. Son dosya: ogrenciNo.zip  
. Başlık: ODEV2  
. Mail adresi: batuhanbardak@etu.edu.tr ve m.turhan@etu.edu.tr  

# Python 3.10.6 64-bit

### exploring data

In [ ]:
import pandas as pd
dataset = pd.read_csv("housing.csv", delimiter=r"\s+", names=["CRIM","ZN","INDUS","CHAS","NOX","RM","AGE","DIS","RAD","TAX","PTRATIO","B","LSTAT","PRICE"], header=None)
dataset = dataset.sample(frac=1, random_state=201401004).reset_index(drop=True)
dataset

In [ ]:
dataset.info()

In [ ]:
dataset.describe()

In [ ]:
import seaborn as sns
ax = sns.heatmap(
    dataset.corr(), 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)

In [ ]:
dataset.hist(layout=(3,5), figsize=(15,10));

In [ ]:
X_train = dataset.iloc[:int(len(dataset)*0.7)][dataset.columns[:-1]]
y_train = dataset.iloc[:int(len(dataset)*0.7)][dataset.columns[-1]]

X_validation = dataset.iloc[int(len(dataset)*0.7):int(len(dataset)*0.9)][dataset.columns[:-1]]
y_validation = dataset.iloc[int(len(dataset)*0.7):int(len(dataset)*0.9)][dataset.columns[-1]]

X_test = dataset.iloc[int(len(dataset)*0.9):][dataset.columns[:-1]]
y_test = dataset.iloc[int(len(dataset)*0.9):][dataset.columns[-1]]

In [ ]:
for col in dataset.columns[:-1]:
    X_train_col_min = X_train[col].min()
    X_train_col_max = X_train[col].max()

    X_train[col] = (X_train[col] - X_train_col_min) / (X_train_col_max - X_train_col_min)
    X_validation[col] = (X_validation[col] - X_train_col_min) / (X_train_col_max - X_train_col_min)
    X_test[col] = (X_test[col] - X_train_col_min) / (X_train_col_max - X_train_col_min)


y_train_col_min = y_train.min()
y_train_col_max = y_train.max()

y_train = (y_train - y_train_col_min) / (y_train_col_max - y_train_col_min)
y_validation = (y_validation - y_train_col_min) / (y_train_col_max - y_train_col_min)
y_test = (y_test - y_train_col_min) / (y_train_col_max - y_train_col_min)

### preprocessing

In [2]:
def set_data():    
    import pandas as pd
    dataset = pd.read_csv("housing.csv", delimiter=r"\s+", names=["CRIM","ZN","INDUS","CHAS","NOX","RM","AGE","DIS","RAD","TAX","PTRATIO","B","LSTAT","PRICE"], header=None)
    dataset = dataset.sample(frac=1, random_state=201401004).reset_index(drop=True)

    X_train = dataset.iloc[:int(len(dataset)*0.7)][dataset.columns[:-1]]
    y_train = dataset.iloc[:int(len(dataset)*0.7)][dataset.columns[-1]]

    X_validation = dataset.iloc[int(len(dataset)*0.7):int(len(dataset)*0.9)][dataset.columns[:-1]]
    y_validation = dataset.iloc[int(len(dataset)*0.7):int(len(dataset)*0.9)][dataset.columns[-1]]

    X_test = dataset.iloc[int(len(dataset)*0.9):][dataset.columns[:-1]]
    y_test = dataset.iloc[int(len(dataset)*0.9):][dataset.columns[-1]]

    for col in dataset.columns[:-1]:
        X_train_col_min = X_train[col].min()
        X_train_col_max = X_train[col].max()

        X_train[col] = (X_train[col] - X_train_col_min) / (X_train_col_max - X_train_col_min)
        X_validation[col] = (X_validation[col] - X_train_col_min) / (X_train_col_max - X_train_col_min)
        X_test[col] = (X_test[col] - X_train_col_min) / (X_train_col_max - X_train_col_min)
        dataset[col] = (dataset[col] - X_train_col_min) / (X_train_col_max - X_train_col_min)


    y_train_col_min = y_train.min()
    y_train_col_max = y_train.max()

    y_train = (y_train - y_train_col_min) / (y_train_col_max - y_train_col_min)
    y_validation = (y_validation - y_train_col_min) / (y_train_col_max - y_train_col_min)
    y_test = (y_test - y_train_col_min) / (y_train_col_max - y_train_col_min)
    dataset[dataset.columns[-1]] = (dataset[dataset.columns[-1]] - y_train_col_min) / (y_train_col_max - y_train_col_min)


    train_dataset = dataset.iloc[:int(len(dataset)*0.8)]
    valid_dataset = dataset.iloc[int(len(dataset)*0.8):int(len(dataset)*0.9)]
    test_dataset = dataset.iloc[int(len(dataset)*0.9):]


    return dataset, train_dataset, valid_dataset, test_dataset, X_train, y_train, X_validation, y_validation, X_test, y_test

In [3]:
dataset, train_dataset, valid_dataset, test_dataset, X_train, y_train, X_validation, y_validation, X_test, y_test = set_data()

In [4]:
train_dataset.to_csv("housing_for_Cpp_train.csv", header=False, index=False)
valid_dataset.to_csv("housing_for_Cpp_valid.csv", header=False, index=False)
test_dataset.to_csv("housing_for_Cpp_test.csv", header=False, index=False)

In [6]:
len(train_dataset), len(valid_dataset), len(test_dataset)

(404, 51, 51)

### creating NN

In [ ]:
class NN():
    def create(self, hidden_layers=[], activation_function="sigmoid", learning_rate=0.005):
        import numpy as np
        # Creating layer weights and biasses
        if len(hidden_layers)!=0:
            self.weights = []
            self.biasses = []
            for (layer1, layer2) in zip(hidden_layers[:-1],hidden_layers[1:]):
                np.random.seed(int(len(hidden_layers) * learning_rate)**2)
                self.weights.append(np.random.uniform(low=-1, high=1, size=(layer1, layer2)))
                np.random.seed(int(len(hidden_layers) * learning_rate)**2)
                self.biasses.append(np.random.uniform(low=-1, high=1, size=(layer2, 1)))   
        else:
            raise Exception("you must give layer sizes")
            
        # declaring activation function
        def RelU(X):
            return np.maximum(0, X)
        def sigmoid(X):
            return np.exp(X)/(1 + np.exp(X))
        def tanh(X):
            return (np.exp(X) - np.exp(-X))/(np.exp(X) + np.exp(-X))
        if activation_function=="RelU":
            self.activation_func = RelU
        elif activation_function=="sigmoid":
            self.activation_func = sigmoid
        elif activation_function=="tanh":
            self.activation_func = tanh
            
        # declaring derivative of activation function
        def derivative_RelU(X):
            X[X<0]=0
            X[X>0]=1
            return X
        def derivative_sigmoid(X):
            return sigmoid(X)*(1-sigmoid(X))
        def derivative_tanh(X):
            return 1 - tanh(X)**2
        if activation_function=="RelU":
            self.derivative_activation_func = derivative_RelU
        elif activation_function=="sigmoid":
            self.derivative_activation_func = derivative_sigmoid
        elif activation_function=="tanh":
            self.derivative_activation_func = derivative_tanh

        self.alpha = learning_rate

    ######---######---######---######---######---######---######---######---######---######---######---######---######---######---######---######---######---######
    def loss(self, actual, pred):
        import numpy as np
        actual = np.array([a[0] for a in actual])
        pred = np.array([p[0] for p in pred])
        differences = actual-pred
        differences_squared = differences*differences
        differences_squared_sum = differences_squared.sum()
        return differences_squared_sum/len(differences)
        
    def derivative_loss(self, actual, pred):
        return (2/len(actual))*((pred-actual).sum())

    ######---######---######---######---######---######---######---######---######---######---######---######---######---######---######---######---######---######
    def forward_prop(self, X):
        import numpy as np
        Zs = []
        As = []
        for weight, bias in zip(self.weights, self.biasses):
            Z = weight.T @ (np.reshape(X, (len(X),1))) + bias
            Zs.append(Z)
            X = self.activation_func(Z)
            As.append(X)
        return Zs, As

    ######---######---######---######---######---######---######---######---######---######---######---######---######---######---######---######---######---######
    def back_prop(self, weights, Zs, As, X, Y):
        import numpy as np
        bias_errors = []
        weight_errors = []
        for layer in range(len(self.weights)-1, -1, -1):
            # calculating error
            if layer == len(self.weights)-1: # output
                error = self.derivative_loss(Y, As[-1]) * self.derivative_activation_func(Zs[-1])
                bias_errors.insert(0, error)
            else: 
                error = (weights[layer+1] @ error) * self.derivative_activation_func(Zs[layer])
                bias_errors.insert(0, error)

            # calculating weight error
            if layer == 0: # input
                weight_errors.insert(0, np.reshape(X, (len(X),1)) @ error.T)
            else:
                weight_errors.insert(0, As[layer-1] @ error.T)

        return bias_errors, weight_errors  
    
    ######---######---######---######---######---######---######---######---######---######---######---######---######---######---######---######---######---######
    def update_parameters(self, alpha, bias_errors, weight_errors):
        new_weights = self.weights.copy()
        new_biasses = self.biasses.copy()
        
        for bi in range(len(new_biasses)):
            new_biasses[bi] = self.biasses[bi] - alpha*bias_errors[bi] 
        for wi in range(len(new_weights)):
            new_weights[wi] = self.weights[wi] - alpha*weight_errors[wi]       

        self.biasses = new_biasses
        self.weights = new_weights

    ######---######---######---######---######---######---######---######---######---######---######---######---######---######---######---######---######---######
    def gradient_descent(self, mini_batch):
        import warnings
        warnings.filterwarnings("ignore")
        import numpy as np
        total_bias_error = None
        total_weight_error = None

        for (X, Y) in mini_batch:
            batch_Zs, batch_As = self.forward_prop(X)
            batch_bias_error, batch_weight_error = self.back_prop(self.weights, batch_Zs, batch_As, X, Y)

            if total_bias_error == None:
                total_bias_error = np.array(batch_bias_error)
            else:
                total_bias_error = np.array(total_bias_error) + np.array(batch_bias_error)
            
            if total_weight_error == None:
                total_weight_error = np.array(batch_weight_error)
            else:
                total_weight_error = np.array(total_weight_error) + np.array(batch_weight_error)
                
        return total_bias_error/len(mini_batch), total_weight_error/len(mini_batch)

    ######---######---######---######---######---######---######---######---######---######---######---######---######---######---######---######---######---######
    def fit(self, epoch, batch_size, X, Y, VX, VY, verbose=1):
        from datetime import datetime
        import numpy as np
        losses = []
        current_learning_rate = self.alpha
        for e in range(epoch):
            if epoch != 0 and e % 100 == 0:
                current_learning_rate = current_learning_rate * 0.25
                
            if verbose==1 and epoch != 0 and e % int(epoch**0.5) == 0:
                print(datetime.now().strftime("%H:%M:%S"), end=" ----- ")
                print("epoch:",e+1, end=" ----- ")

            SEED = np.random.randint(0, 201401004)
            np.random.seed(SEED)
            np.random.shuffle(X)
            np.random.seed(SEED)
            np.random.shuffle(Y)
            np.random.seed(SEED)
            np.random.shuffle(VX)
            np.random.seed(SEED)
            np.random.shuffle(VY)

            train_data = [(x_row, y) for x_row,  y in zip(X,Y)]
            validation_data = [(x_row, y) for x_row,  y in zip(VX,VY)]

            batch_number = np.floor(len(train_data)/batch_size)
            batch_sizes = [batch_size]*int(batch_number)
            if batch_size*batch_number != len(train_data):
                batch_sizes.append(int(len(train_data) - batch_size*batch_number))
            batches = np.split(train_data, np.cumsum(batch_sizes))[:-1]

            for b,batch in enumerate(batches):
                batch_bias_error, batch_weight_error = self.gradient_descent(batch)
                self.update_parameters(current_learning_rate, batch_bias_error, batch_weight_error)

            epoch_train_loss = self.loss(np.array(train_data)[:,1], self.predict(np.array(train_data)[:,0]))
            epoch_validation_loss = self.loss(np.array(validation_data)[:,1], self.predict(np.array(validation_data)[:,0]))
            losses.append((epoch_train_loss, epoch_validation_loss))
            if verbose==1 and epoch != 0 and e % int(epoch**0.5) == 0:
                print("train loss:", epoch_train_loss, "-----" ,"validation loss:", epoch_validation_loss)

        return losses
        
    ######---######---######---######---######---######---######---######---######---######---######---######---######---######---######---######---######---######
    def predict(self, X):
        predictions = []
        for x in X:
            _, As = self.forward_prop(x)
            predictions.append(As[-1][0])
        return predictions

### Gizli katman sayısının 0, 1, ve 2 olduğundaki eğitim ve test hata fonksiyonlarını çizdirerek gösteriniz.  

In [ ]:
def hidden_layer_test(layers):
    X_train, y_train, X_validation, y_validation, X_test, y_test = set_data()
    import numpy as np
    X = np.reshape(np.array(X_train), (len(X_train), len(X_train.iloc[0])))
    Y = np.reshape(np.array(y_train), (len(y_train), 1))
    X_v = np.reshape(np.array(X_validation), (len(X_validation), len(X_validation.iloc[0])))
    Y_v = np.reshape(np.array(y_validation), (len(y_validation), 1))
    X_t = np.reshape(np.array(X_test), (len(X_test), len(X_test.iloc[0])))
    Y_t = np.reshape(np.array(y_test), (len(y_test), 1))

    import warnings
    warnings.filterwarnings("ignore")
    nn = NN()
    nn.create(layers, "sigmoid", 0.01)
    initial_prediction = nn.predict(X_v)

    losses = nn.fit(250, 1, X, Y, X_v, Y_v)
    learned_prediction = nn.predict(X_v)

    import seaborn as sns
    import matplotlib.pyplot as plt

    # plot lines
    plt.plot(range(len(losses)), [v for (t,v) in losses], label = "validation error")
    plt.plot(range(len(losses)), [t for (t,v) in losses], label = "train error")
    plt.legend()
    plt.show()

    trues = [y[0] for y in Y_v]
    trues.sort()
    initials = [p[0] for p in initial_prediction]
    initials.sort()
    predictions = [p[0] for p in learned_prediction]
    predictions.sort()

    # plot lines
    plt.plot(range(len(Y_v)), trues, label = "true")
    plt.plot(range(len(Y_v)), initials, label = "validation predictions before fit")
    plt.plot(range(len(Y_v)), predictions, label = "validation predictions after fit")
    plt.legend()
    plt.show()


    test_predictions = nn.predict(X_t)
    import matplotlib.pyplot as plt
    trues = [y[0] for y in Y_t]
    trues.sort()
    test_prediction = [p[0] for p in test_predictions]
    test_prediction.sort()

    # plot lines
    plt.plot(range(len(Y_t)), trues, label = "true")
    plt.plot(range(len(Y_t)), test_prediction, label = "test predictions")
    plt.legend()
    plt.show()

    return nn.loss(Y_t, nn.predict(X_t))

#### gizli katman sayısı 0

In [ ]:
test_loss = hidden_layer_test([13,1])
print("test loss:", test_loss)

#### gizli katman sayısı 1

In [ ]:
test_loss = hidden_layer_test([13, 26, 1])
print("test loss:", test_loss)

#### gizli katman sayısı 2

In [ ]:
test_loss = hidden_layer_test([13, 26, 13, 1])
print("test loss:", test_loss)

### Learning rate parametresinin (0.1, 0.01 vb.) değiştirilmesinin eğitim hatasını çizdirerek gösteriniz. 

In [ ]:
def learning_rate_test(lr):
    X_train, y_train, X_validation, y_validation, X_test, y_test = set_data()
    import numpy as np
    X = np.reshape(np.array(X_train), (len(X_train), len(X_train.iloc[0])))
    Y = np.reshape(np.array(y_train), (len(y_train), 1))
    X_v = np.reshape(np.array(X_validation), (len(X_validation), len(X_validation.iloc[0])))
    Y_v = np.reshape(np.array(y_validation), (len(y_validation), 1))
    X_t = np.reshape(np.array(X_test), (len(X_test), len(X_test.iloc[0])))
    Y_t = np.reshape(np.array(y_test), (len(y_test), 1))

    import warnings
    warnings.filterwarnings("ignore")
    nn = NN()
    nn.create([13, 26, 1], "sigmoid", lr)
    initial_prediction = nn.predict(X)

    losses = nn.fit(250, 1, X, Y, X_v, Y_v)
    learned_prediction = nn.predict(X)

    import seaborn as sns
    import matplotlib.pyplot as plt

    # plot lines
    plt.plot(range(len(losses)), [v for (t,v) in losses], label = "validation error")
    plt.plot(range(len(losses)), [t for (t,v) in losses], label = "train error")
    plt.legend()
    plt.show()

    trues = [y[0] for y in Y]
    trues.sort()
    initials = [p[0] for p in initial_prediction]
    initials.sort()
    predictions = [p[0] for p in learned_prediction]
    predictions.sort()

    # plot lines
    plt.plot(range(len(Y)), trues, label = "true")
    plt.plot(range(len(Y)), initials, label = "train predictions before fit")
    plt.plot(range(len(Y)), predictions, label = "train predictions after fit")
    plt.legend()
    plt.show()


    test_predictions = nn.predict(X_t)
    import matplotlib.pyplot as plt
    trues = [y[0] for y in Y_t]
    trues.sort()
    test_prediction = [p[0] for p in test_predictions]
    test_prediction.sort()

    # plot lines
    plt.plot(range(len(Y_t)), trues, label = "true")
    plt.plot(range(len(Y_t)), test_prediction, label = "test predictions")
    plt.legend()
    plt.show()

    return nn.loss(Y_t, nn.predict(X_t))

#### learning rate = 0.1

In [ ]:
test_loss = learning_rate_test(0.1)
print("test loss:", test_loss)

#### learning rate = 0.01

In [ ]:
test_loss = learning_rate_test(0.01)
print("test loss:", test_loss)

#### learning rate = 0.001

In [ ]:
test_loss = learning_rate_test(0.001)
print("test loss:", test_loss)

### Katmanlardaki nöron sayısının değişiminin, eğitim ve test hatalarına etkisini gözlemleyiniz.

In [ ]:
def neuron_number_test(neuron_number):
    X_train, y_train, X_validation, y_validation, X_test, y_test = set_data()
    import numpy as np
    X = np.reshape(np.array(X_train), (len(X_train), len(X_train.iloc[0])))
    Y = np.reshape(np.array(y_train), (len(y_train), 1))
    X_v = np.reshape(np.array(X_validation), (len(X_validation), len(X_validation.iloc[0])))
    Y_v = np.reshape(np.array(y_validation), (len(y_validation), 1))
    X_t = np.reshape(np.array(X_test), (len(X_test), len(X_test.iloc[0])))
    Y_t = np.reshape(np.array(y_test), (len(y_test), 1))

    import warnings
    warnings.filterwarnings("ignore")
    nn = NN()
    nn.create([13, neuron_number, 1], "sigmoid", 0.1)
    initial_prediction = nn.predict(X)

    losses = nn.fit(250, 1, X, Y, X_v, Y_v)
    learned_prediction = nn.predict(X)

    import seaborn as sns
    import matplotlib.pyplot as plt

    # plot lines
    plt.plot(range(len(losses)), [v for (t,v) in losses], label = "validation error")
    plt.plot(range(len(losses)), [t for (t,v) in losses], label = "train error")
    plt.legend()
    plt.show()

    trues = [y[0] for y in Y]
    trues.sort()
    initials = [p[0] for p in initial_prediction]
    initials.sort()
    predictions = [p[0] for p in learned_prediction]
    predictions.sort()

    # plot lines
    plt.plot(range(len(Y)), trues, label = "true")
    plt.plot(range(len(Y)), initials, label = "train predictions before fit")
    plt.plot(range(len(Y)), predictions, label = "train predictions after fit")
    plt.legend()
    plt.show()


    test_predictions = nn.predict(X_t)
    import matplotlib.pyplot as plt
    trues = [y[0] for y in Y_t]
    trues.sort()
    test_prediction = [p[0] for p in test_predictions]
    test_prediction.sort()

    # plot lines
    plt.plot(range(len(Y_t)), trues, label = "true")
    plt.plot(range(len(Y_t)), test_prediction, label = "test predictions")
    plt.legend()
    plt.show()

    return nn.loss(Y_t, nn.predict(X_t))

#### 30 neuron

In [ ]:
test_loss = neuron_number_test(30)
print("test loss:", test_loss)

#### 60 neuron

In [ ]:
test_loss = neuron_number_test(60)
print("test loss:", test_loss)

#### 90 neuron

In [ ]:
test_loss = neuron_number_test(90)
print("test loss:", test_loss)

### Epoch ve eğitim/validasyon hatası arasındaki ilişkiyi gösteren figür çizdiriniz.  

In [ ]:
X_train, y_train, X_validation, y_validation, X_test, y_test = set_data()
import numpy as np
X_train = np.reshape(np.array(X_train), (len(X_train), len(X_train.iloc[0])))
y_train = np.reshape(np.array(y_train), (len(y_train), 1))
X_validation = np.reshape(np.array(X_validation), (len(X_validation), len(X_validation.iloc[0])))
y_validation = np.reshape(np.array(y_validation), (len(y_validation), 1))
X_test = np.reshape(np.array(X_test), (len(X_test), len(X_test.iloc[0])))
y_test = np.reshape(np.array(y_test), (len(y_test), 1))

In [ ]:
import warnings
warnings.filterwarnings("ignore")
nn = NN()
nn.create([13, 30, 1], "sigmoid", 0.1)

In [ ]:
initial_prediction = nn.predict(X_validation)
from sklearn.metrics import mean_squared_error
print("NN mean squared error:", mean_squared_error(y_validation, np.array(initial_prediction), squared=True))
from sklearn.metrics import mean_squared_error
print("NN root mean squared error:", mean_squared_error(y_validation, np.array(initial_prediction) ,squared=False))
from sklearn.metrics import mean_absolute_error
print("NN mean absolute error:", mean_absolute_error(y_validation, np.array(initial_prediction)))
from sklearn.metrics import r2_score
print("NN r2 score:", r2_score(y_validation, np.array(initial_prediction)))
#Adjusted R Squared:
print("NN adjusted r2 score:", 1-(1-r2_score(y_validation, np.array(initial_prediction)))*((len(y_validation))-1)/((len(y_validation))-6))

In [ ]:
losses = nn.fit(500, 1, X_train, y_train, X_validation, y_validation)
import seaborn as sns
import matplotlib.pyplot as plt

# plot lines
plt.plot(range(len(losses)), [v for (t,v) in losses], label = "validation error")
plt.plot(range(len(losses)), [t for (t,v) in losses], label = "train error")
plt.legend()
plt.show()

In [ ]:
learned_prediction = nn.predict(X_validation)
from sklearn.metrics import mean_squared_error
print("NN mean squared error:", mean_squared_error(y_validation, np.array(learned_prediction), squared=True))
from sklearn.metrics import mean_squared_error
print("NN root mean squared error:", mean_squared_error(y_validation, np.array(learned_prediction) ,squared=False))
from sklearn.metrics import mean_absolute_error
print("NN mean absolute error:", mean_absolute_error(y_validation, np.array(learned_prediction)))
from sklearn.metrics import r2_score
print("NN r2 score:", r2_score(y_validation, np.array(learned_prediction)))
#Adjusted R Squared:
print("NN adjusted r2 score:", 1-(1-r2_score(y_validation, np.array(learned_prediction)))*((len(y_validation))-1)/((len(y_validation))-6))

In [ ]:
# importing package
import matplotlib.pyplot as plt

trues = [y[0] for y in y_validation]
trues.sort()
initials = [p[0] for p in initial_prediction]
initials.sort()
predictions = [p[0] for p in learned_prediction]
predictions.sort()

# plot lines
plt.plot(range(len(y_validation)), trues, label = "true")
plt.plot(range(len(y_validation)), initials, label = "validation predictions before fit")
plt.plot(range(len(y_validation)), predictions, label = "validation predictions after fit")
plt.legend()
plt.show()

In [ ]:
test_predictions = nn.predict(X_test)

import matplotlib.pyplot as plt
trues = [y[0] for y in y_test]
trues.sort()
test_prediction = [p[0] for p in test_predictions]
test_prediction.sort()

# plot lines
plt.plot(range(len(y_test)), trues, label = "true")
plt.plot(range(len(y_test)), test_prediction, label = "test predictions")
plt.legend()
plt.show()

### grid search

In [ ]:
import warnings
warnings.filterwarnings("ignore")

X_train, y_train, X_validation, y_validation, X_test, y_test = set_data()
import numpy as np
X_train = np.reshape(np.array(X_train), (len(X_train), len(X_train.iloc[0])))
y_train = np.reshape(np.array(y_train), (len(y_train), 1))
X_validation = np.reshape(np.array(X_validation), (len(X_validation), len(X_validation.iloc[0])))
y_validation = np.reshape(np.array(y_validation), (len(y_validation), 1))
X_test = np.reshape(np.array(X_test), (len(X_test), len(X_test.iloc[0])))
y_test = np.reshape(np.array(y_test), (len(y_test), 1))

In [ ]:
layers = [[13, 24, 48, 24, 16, 1],
          [13, 16, 32, 16, 1],
          [13, 26, 1]]
activation_funcs = ["sigmoid", "tanh", "RelU"]
learning_rates = [0.1, 0.05]
epochs = [250, 500, 750]
batchs = [1, 4, 7]

In [ ]:
import itertools
grid_losses = []
grid_weights = []
grid_biasses = []
grid_test_predictions = []
grid_test_losses = []
for i, parameters in enumerate(itertools.product(layers, activation_funcs, learning_rates, epochs, batchs)):
    print(i," out of ", (len(layers)*len(activation_funcs)*len(learning_rates)*len(epochs)*len(batchs)), " NN learning with ", parameters, sep="")
    grid_nn = NN()
    grid_nn.create(parameters[0], parameters[1], parameters[2])
    grid_losses.append((parameters, grid_nn.fit(parameters[3], parameters[4], X_train, y_train, X_validation, y_validation, verbose=1)))
    grid_weights.append(grid_nn.weights)
    grid_biasses.append(grid_nn.biasses)
    grid_prediction_i = grid_nn.predict(X_test)
    grid_test_losses.append(grid_nn.loss(y_test, grid_prediction_i))
    grid_prediction_i.sort()
    grid_test_predictions.append(grid_prediction_i)

#### all grid search plots

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
trues = [y[0] for y in y_test]
trues.sort()

for i in range(len(grid_test_predictions)):
    plt.figure()
    plt.tight_layout()
    sns.lineplot(x=range(len(y_test)), y=[p[0] for p in grid_test_predictions[i]], label = "test prediction").set(title="model" + str(i) + ": " + str(grid_losses[i][0]) + " test loss:" + str(grid_test_losses[i]));
    sns.lineplot(x=range(len(y_test)), y=trues, label = "true");

### best parameters

In [ ]:
grid_summary = [(l, w, b, tp, tl) for l, w, b, tp, tl in zip(grid_losses, grid_weights, grid_biasses, grid_test_predictions ,grid_test_losses)]
grid_summary.sort(key = lambda x: x[4])
best_model_summary = grid_summary[0]

In [ ]:
print("""
1- Kullanılan katman sayısı:""", len(best_model_summary[0][0][0]),"""
2- Her katmandaki nöron sayısı:""", best_model_summary[0][0][0],"""
3- Katmanlarda kullanılan aktivasyon fonksiyonu:""", best_model_summary[0][0][1],"""
4- Ağırlıkların ilk değerinin nasıl tanımlandığı: np.random.uniform(low=-1, high=1 size=<weight matrix shape>)
5- Epoch Sayısı:""", best_model_summary[0][0][3],"""
6- Normalizasyon kullanılıp, kullanılmadığı bilgisi: Kullanıldı
7- Batch büyüklüğü:""", best_model_summary[0][0][4],"""
8- Eğitim hatası(train loss, validation loss):""", best_model_summary[0][1][-1],"""
9- Test hatası:""", best_model_summary[4],"""
""")

In [ ]:
plt.figure()
plt.tight_layout()
sns.lineplot(x=range(len(y_test)), y=[p[0] for p in best_model_summary[3]], label = "test prediction").set(title="model" + str(i) + ": " + str(best_model_summary[0][0]) + " test loss:" + str(best_model_summary[4]));
sns.lineplot(x=range(len(y_test)), y=trues, label = "true");